# Ejemplo de `word2vec` con `gensim`


En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [1]:
import gensim, logging, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Entrenamiento de un modelo

Implemento una clase `Corpus` con un iterador sobre un directorio que contiene ficheros de texto. Utilizaré una instancia de `Corpus` para poder procesar de manera más eficiente una colección, sin necesidad de cargarlo previamente en memoria.

In [2]:
class Corpus(object):
    '''Clase Corpus que permite leer de manera secuencial un directorio de documentos de texto'''
    
    def __init__(self, directorio):
        self.directory = directorio

    def __iter__(self):
        for fichero in os.listdir(self.directory):
            for linea in open(os.path.join(self.directory, fichero)):
                yield linea.split()


`CORPUSDIR` contiene una colección de noticias en español (normalizada previamente a minúsculas y sin signos de puntuación) con alrededor de 150 millones de palabras. Entrenamos un modelo en un solo paso, ignorando aquellos tokens que aparecen menos de 10 veces, para descartar erratas. 

In [3]:
CORPUSDIR = '/opt/textos/efe/txt/'
oraciones = Corpus(CORPUSDIR)
#model = gensim.models.Word2Vec(oraciones, min_count=10, size=150, workers=2)

# el modelo puede entrenarse en dos pasos sucesivos pero por separado
#model = gensim.models.Word2Vec() # modelo vacío
#model.build_vocab(oraciones)  # primera pasada para crear la lista de vocabulario
#model.train(other_sentences)  # segunda pasada para calcula vectores

Una vez completado el entrenamiento (después de casi 30 minutos), guardamos el modelo en disco. 

In [ ]:
#model.save('/opt/textos/efe/efe.model.w2v')

En el futuro, podremos utilizar este modelo cargándolo en memoria con la instrucción:

In [4]:
model = gensim.models.Word2Vec.load('/opt/textos/efe/efe.model.w2v')

## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

En nuestro modelo, tal y como está entrenado, tenemos más de 26 millones de términos:

In [5]:
print(model.corpus_count)

26522244


Cada término del vocabulario está representado como un vector con 150 dimensiones: 105 características. Podemos acceder al vector de un término concreto:

In [6]:
print(model['azul'], '\n')

print(model['verde'], '\n')

print(model['microsoft'])

[-0.17119461 -0.22743705  0.33344841  0.27989009  0.19492076 -0.1409657
 -0.0251887  -0.306355    0.10662927  0.17104091 -0.22948962  0.36396649
 -0.23878925 -0.21033089  0.19784033 -0.61557579 -0.21383885  0.44664291
 -0.64749044 -0.2872344  -0.55847257  0.1303734   0.03406142 -0.31790459
  0.21620856  0.24226433  0.27817437  0.26362386 -0.24987099  0.11778339
 -0.47454187 -0.23323169 -0.30103257 -0.03067073 -0.04180949 -0.32016489
 -0.18483758  0.08247121 -0.19331424  0.40855029 -0.47981584  0.16826837
 -0.41814953 -0.4381167  -0.16309766  0.0599881   0.0454194  -0.14878696
  0.47207984  0.26696566 -0.39615464  0.05608032  0.03534137 -0.07457943
  0.10052393 -0.30167556 -0.02699662  0.28504115 -0.404071   -0.44440794
 -0.36049092 -0.13775028  0.31180441  0.33074516 -0.17758727 -0.20152695
  0.18360205  0.36311045  0.11820537  0.17505574 -0.26084182  0.22973292
  0.14244203 -0.21329062  0.24214892 -0.05452183  0.53469735  0.4316895
  0.17438972  0.14840761  0.17512347 -0.06805158 -0.0

Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [7]:
print('hombre - mujer', model.similarity('hombre', 'mujer'))

print('madrid - parís', model.similarity('madrid', 'parís'))

print('perro - gato', model.similarity('perro', 'gato'))

print('gato - periódico', model.similarity('gato', 'periódico'))

hombre - mujer 0.473325617835
madrid - parís 0.649728730699
perro - gato 0.631910988958
gato - periódico 0.180082365746


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [8]:
lista1 = 'madrid barcelona gonzález washington'.split()
print('en la lista', ' '.join(lista1), 'sobra:', model.doesnt_match(lista1))

lista2 = 'psoe pp ciu epi'.split()
print('en la lista', ' '.join(lista2), 'sobra:', model.doesnt_match(lista2))

lista3 = 'publicaron declararon soy negaron'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.doesnt_match(lista3))

lista3 = 'homero saturno cervantes shakespeare cela'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.doesnt_match(lista3))

en la lista madrid barcelona gonzález washington sobra: gonzález
en la lista psoe pp ciu epi sobra: epi
en la lista publicaron declararon soy negaron sobra: soy
en la lista homero saturno cervantes shakespeare cela sobra: saturno


Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [9]:
terminos = 'psoe chicago sevilla aznar podemos estuvieron'.split()

for t in terminos:
    print(t, '==>', model.most_similar(t), '\n')

psoe ==> [('pp', 0.8361572623252869), ('psc', 0.7607941031455994), ('psc-psoe', 0.7428343892097473), ('pnv', 0.7384518384933472), ('pse-ee', 0.7288982272148132), ('pp-', 0.7149612307548523), ('ciu', 0.7143486142158508), ('pce', 0.7000882625579834), ('iu', 0.6951525807380676), ('pse-psoe', 0.6903963685035706)] 

chicago ==> [('boston', 0.7728167772293091), ('filadelfia', 0.7359776496887207), ('dallas', 0.7138493061065674), ('detroit', 0.6884219646453857), ('houston', 0.6674472689628601), ('minneapolis', 0.6521533131599426), ('indianapolis', 0.6486193537712097), ('illinois', 0.6469548940658569), ('massachusetts', 0.6462246775627136), ('minnesota', 0.6423684358596802)] 

sevilla ==> [('valencia', 0.8517122268676758), ('oviedo', 0.828134298324585), ('valladolid', 0.8251886963844299), ('madrid', 0.8147549629211426), ('barcelona', 0.8145360946655273), ('mérida', 0.8092716932296753), ('toledo', 0.8011802434921265), ('badajoz', 0.791728675365448), ('albacete', 0.7854650616645813), ('málaga', 0

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [10]:
print('==> alcalde + mujer - hombre')
most_similar = model.most_similar(positive=['alcalde', 'mujer'], negative=['hombre'], topn=3)
for item in most_similar:
    print(item)

print('==> madrid + filipinas - españa')
most_similar = model.most_similar(positive=['madrid', 'filipinas'], negative=['españa'], topn=3)
for item in most_similar:
    print(item)

print('==> michel + fútbol + argentina - españa')
most_similar = model.most_similar(positive=['michel', 'fútbol', 'argentina'], negative=['españa'], topn=3)
for item in most_similar:
    print(item)

==> alcalde + mujer - hombre
('alcaldesa', 0.5925700068473816)
('consellera', 0.5343092679977417)
('concejala', 0.5338941812515259)
==> madrid + filipinas - españa
('manila', 0.5159924030303955)
('pekín', 0.5065802335739136)
('hanoi', 0.49402812123298645)
==> michel + fútbol + argentina - españa
('platini', 0.529076099395752)
('ex-capitán', 0.5200508236885071)
('fpf', 0.4923384487628937)
